# aliframework GCS + BigQuery demo (requires real GCP project)

In [ ]:
from aliframework.config import GcpConfig
from aliframework.gcp import (
    create_gcs_client,
    create_bigquery_client,
    gcs_to_gcs,
    gcs_to_pandas,
    pandas_to_gcs,
    gcs_to_bq,
    df_to_bq,
    bq_to_gcs,
    gcs_to_df,
)
import pandas as pd

# Set your own project, bucket, dataset, table before running
PROJECT_ID = 'your-project'
BUCKET = 'your-bucket'
DATASET = 'your_dataset'
TABLE = 'your_table'

cfg = GcpConfig(project_id=PROJECT_ID, credentials_path=None)
gcs_client = create_gcs_client(cfg)
bq_client = create_bigquery_client(cfg)

src_uri = f'gs://{BUCKET}/input.csv'
dst_uri = f'gs://{BUCKET}/copy.csv'
gcs_to_gcs(gcs_client, src_uri, dst_uri)
df = gcs_to_pandas(gcs_client, dst_uri)
print('rows read:', len(df))
out_uri = f'gs://{BUCKET}/from_df.csv'
pandas_to_gcs(gcs_client, df, out_uri, pandas_to_fn='to_csv', index=False)

table_id = f'{PROJECT_ID}.{DATASET}.{TABLE}'
gcs_to_bq(bq_client, table_id=table_id, source_uri=out_uri, file_format='CSV', autodetect=True, skip_leading_rows=1)
df2 = pd.DataFrame([{'id': 999, 'name': 'from_notebook'}])
df_to_bq(bq_client, df2, table_id=table_id)

export_uri = f'gs://{BUCKET}/export-*.csv'
bq_to_gcs(bq_client, table_id=table_id, destination_uri=export_uri, file_format='CSV')
print('Exported to', export_uri)
